# Exercise 5 – Interpretation von Modellen

### a) Lesen Sie den Datensatz Hdma.csv ein und machen Sie sich mit den Daten vertraut.

Einlesen des Datensatzes

In [24]:
import pandas as pd

# load dataset
data = pd.read_csv('./data/Hdma.csv', sep=';')

Vertrautmachen mit den Daten

In [25]:
data.head()

,dir,hir,lvr,ccs,mcs,pbcr,dmi,self,single,uria,condo,black,deny
0,0.221,0.221,0.800000,5.0,2.0,no,no,no,no,3.9,0,no,no
1,0.265,0.265,0.921875,2.0,2.0,no,no,no,yes,3.2,0,no,no
2,0.372,0.248,0.920398,1.0,2.0,no,no,no,no,3.2,0,no,no
3,0.320,0.250,0.860465,1.0,2.0,no,no,no,no,4.3,0,no,no
4,0.360,0.350,0.600000,1.0,1.0,no,no,no,no,3.2,0,no,no


In [22]:
data.max()

dir          3
hir          3
lvr       1.95
ccs          6
mcs          4
dmi        yes
single     yes
uria      10.6
condo        1
black      yes
deny       yes
dtype: object

### b) Volkswirtschaftler möchten herausfinden, was die möglichen Einflussfaktoren auf die Ablehnung eines Kreditantrags sind, und ob es eine rassistische Diskriminierung gibt. Welcher Typ von Aufgabe des maschinellen Lernens ist für diese Aufgabe geeignet? Welche Methoden kann man benutzen?